# PyMem3DG Tutorial

In [1]:
import pymem3dg

## System initialization

### Set up parameters
Set up the parameters needed to specify the physics of the system. The numbers of 0 and -1 are utilized to disable certain features, which will be clearer when combined with option specifications. The underlying system construction will also generally check conflicting parameters.

In [2]:
p = pymem3dg.Parameters()

# Bending Rigidity 
p.Kb = 8.22e-5 
# Spontaneous curvture
p.H0 = 0
# Specific point on the mesh
p.pt = [0, 0, 0]
# Radius of local curvature
p.r_H0 = [-1, -1]
# Sharpness of heterogenous curvature transition
p.sharpness = 0 
# Line tension
p.eta = 0

# Tension
p.Ksg = 1

# Conformal regularization
p.Kst = 7
# Local face regularization
p.Ksl = 0
# Edge regularization
p.Kse = 0

# Binding energy per protein
p.epsilon = -1
# Binding rate constant
p.Bc = -1

# Omostic pressure-volume relation
p.Kv = 0.1
# Target reduced volume
p.Vt = -1
# Ambient osmotic concentration
p.cam = 0.3

# External force
p.Kf = 0
# Concentration of force
p.conc = -1
# Target height of springlike external force
p.height = 0

# Radius of integration
p.radius = 100000

# Damping coefficient of DPD
p.gamma = 0.1
# Temperature
p.temp = 0.1

# Augmented Lagragian coefficients
p.lambdaSG = 0
p.lambdaV = 0

### Option specification
Here we specifying path to the input mesh and its reference configuration, which in the .ply file format. Caution that the input mesh and its reference mesh must have the same topology, meaning that the number of vertices, edges and faces, and their connectivity are the same. They only differ in the realization of geometry, essentially the vertex coordinates. Here we also specify the number of subdivision to determine the fineness  of the mesh, as well as further options for the simulation.

ALternatively, if for continuation, we could take mesh input from netcdf trajectory file, where additionally we have to provide the index of frame as the starting input mesh, and whether or not continue the simulation. The option of continuation will map the variable read from netcdf file based on parameter above and options below.

In [4]:
# inputMesh = "C://Users//Kieran//Dev//2020-Mem3DG-Applications//run//input-file//slightlyOblate.ply"
# refMesh =  "C://Users//Kieran//Dev//2020-Mem3DG-Applications//run//input-file//slightlyOblate.ply"
inputMesh = "/home/cuzhu/2020-Mem3DG-Applications/run/input-file/slightlyOblate.ply"
refMesh = "/home/cuzhu/2020-Mem3DG-Applications/run/input-file/slightlyOblate.ply"

# number of subdivision
nSub = 0

# Whether considering protein binding
isProtein = False
# Whether shift vertex to barycenter for regularization
isVertexShift = False
# Whether use the reduced volume formulation, or if false, ambient concentration model
isReducedVolume = False
# Whether need local curvature formulation, which requires the more expensive computation of geodesic curvature
isLocalCurvature = False

trajFile = ""
startingFrame = 3
isContinue = False

### Instantiate System object
With the preparation above, we could finally construct the System objects

In [5]:
f = pymem3dg.System(inputMesh, refMesh, nSub, p, isReducedVolume,
                    isProtein, isLocalCurvature, isVertexShift)
#f = pymem3dg.System(trajFile, startingFrame, nSub, p, isReducedVolume,isProtein, isLocalCurvature, isVertexShift)

### Computation and inspection of energy and forces
We may inspect various data of the system such as the parameters and their derived feature, such as surface area and enclosed volume of the mesh. More importantly, we can compute each components of energy and subsequently the force to the system. The exaustive list of support quantities will be listed in documentation.

In [12]:
f.computeFreeEnergy()
print("Bending energy is: ", f.E.BE) 
print("\nSurface energy is: ", f.E.sE)
print("\nTotal energy is: ", f.E.totalE)

f.computeBendingPressure()
f.computeAllForces()
print("\nBending vertex Pressure is: \n", f.getBendingPressure())
print("\nOsmotic Pressure is: ", f.getInsidePressure())

Bending energy is:  0.0010292242662725644

Surface energy is:  0.0

Total energy is:  0.0035449070965354974

Bending vertex Pressure is: 
 [[ 5.90731564e-07  5.74377116e-06  2.24537416e-05]
 [-1.67335071e-06 -2.64163732e-06 -7.81694673e-06]
 [-4.00675461e-05 -2.07464984e-04 -4.74728323e-04]
 ...
 [ 3.50158959e-07  1.91919431e-07  1.33877366e-06]
 [-2.37646595e-06  8.51556719e-06  3.63778994e-05]
 [ 5.17788342e-06  1.02066572e-05  5.47380918e-05]]

Osmotic Pressure is:  -0.005833396050151891


## Integrator initialization
To evolve the system, there are three types of propagators to choose in pymem3dg. Velocity Verlet is used to quanlidatively represent the dynamics of membrane dynamics, and can be coupled with Dissipative Particle Dynamics. Conjugate Gradient is used to find the minimizer of the total free energy. Forward Euler method used for rapid prototyping, physically representing the overdamped system and in optimization term a simple gradient descent method.   

In [24]:
# time step for integration
h = 0.001
# total simulation time, or time limit for optimization
T = 1
# Tolerance for termination, measuring the magnitude of overall force to the system
eps = 1e-6
# period of saving output data
tSave = 1
# whether scale time step according to the current mesh size
isAdaptiveStep = True
# path to the output directory
# outputDir = "C://Users//Kieran//Dev//2020-Mem3DG-Applications//results//vesicle//testAPI"
outputDir = "/home/cuzhu/2020-Mem3DG-Applications/results/vesicle/testAPI"

# whether utilizing the backtracking line search algorithm
isBacktrack = True
# Backtracking coefficient
rho = 0.5
# Wolfe condition coefficient
c1 = 0

# verbosity of the integration. The specification can be found on documentation
verbosity = 3

# whether utilize augmented lagragian model for optimization
isAugmentedLagrangian = False


Physical Parameters may be incompatible with the specfic integrator. For example, we cannot have introduce the DPD damping in conjugate gradient or euler integration

In [25]:
f.P.gamma = 0
f.P.temp = 0
cg = pymem3dg.ConjugateGradient(f, h, isAdaptiveStep, T,
                                tSave, eps, outputDir, "/traj.nc", verbosity, isBacktrack, rho, c1, 0.01, isAugmentedLagrangian)
cg.integrate()

False

Similarly, the calculation of DPD need to be coupled with the magnitude of time step. Velocity Verlet integrate will throw error when detected not consistent value. 

In [26]:
f.P.gamma = 0.1
f.P.temp = 0.1
f.P.sigma = (2 * f.P.gamma * 1.380649e-8 * f.P.temp / h)**0.5
vv = pymem3dg.VelocityVerlet(f, h, isAdaptiveStep, T,
                             tSave, eps, outputDir, "/traj4.nc", verbosity)
vv.integrate()
cg.status()
cg.saveData()
cg.march()
cg.step(100)

f.visualize()

RuntimeError: [polyscope] ERROR: Failed to initialize glfw